# Day16_0: PyTorch 기초 (Tensor & Autograd)

## 학습 목표

**Part 1: 기초**
1. PyTorch 텐서 생성 및 기본 연산 이해하기
2. 텐서의 shape, dtype, device 속성 다루기
3. NumPy와 PyTorch 텐서 변환하기
4. GPU 가속(CUDA) 개념 이해하기
5. 텐서 인덱싱과 슬라이싱 활용하기

**Part 2: 심화**
1. Autograd(자동 미분) 원리 이해하기
2. requires_grad와 backward() 활용하기
3. nn.Module로 신경망 정의하기
4. Dataset과 DataLoader로 데이터 공급하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| Tensor | 딥러닝 데이터 표현 | 이미지, 텍스트, 시계열 데이터 |
| Autograd | 자동 그래디언트 계산 | 역전파 학습 자동화 |
| nn.Module | 신경망 구조 정의 | MLP, CNN, RNN 모델 설계 |
| DataLoader | 효율적 데이터 공급 | 배치 처리, 셔플링 |

**분석가 관점**: PyTorch는 연구와 프로덕션 모두에서 가장 인기 있는 딥러닝 프레임워크입니다. 직관적인 Python 스타일과 동적 계산 그래프로 디버깅이 쉽습니다!

---

# Part 1: 기초

---

## 1.1 PyTorch 소개

### 딥러닝 프레임워크 비교

| 구분 | PyTorch | TensorFlow |
|------|---------|------------|
| 개발사 | Meta (Facebook) | Google |
| 계산 그래프 | 동적 (Define-by-Run) | 정적 (TF1) / 동적 (TF2) |
| 디버깅 | Python 스타일 (쉬움) | 상대적으로 복잡 |
| 연구 | 학계에서 선호 | 산업에서 선호 |
| 장점 | 직관적, 빠른 프로토타입 | 대규모 배포 용이 |

In [ ]:
# PyTorch 설치 확인
import torch
print(f"PyTorch 버전: {torch.__version__}")

# CUDA (GPU) 사용 가능 여부
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
else:
    print("CPU 모드로 실행됩니다.")

: 

---

## 1.2 텐서 생성

### 텐서(Tensor)란?

- 다차원 배열 (NumPy의 ndarray와 유사)
- GPU에서 연산 가능
- 자동 미분 지원

```
스칼라(0D) -> 벡터(1D) -> 행렬(2D) -> 텐서(3D+)
    5         [1,2,3]     [[1,2],[3,4]]   [[[1,2],[3,4]],[[5,6],[7,8]]]
```

In [ ]:
# 1. torch.tensor() - 직접 값 지정
scalar = torch.tensor(5)               # 스칼라 (0차원)
vector = torch.tensor([1, 2, 3])       # 벡터 (1차원)
matrix = torch.tensor([[1, 2], [3, 4]])  # 행렬 (2차원)

print(f"스칼라: {scalar}, shape: {scalar.shape}")
print(f"벡터: {vector}, shape: {vector.shape}")
print(f"행렬:\n{matrix}, shape: {matrix.shape}")

In [ ]:
# 2. 특수 텐서 생성 함수
zeros = torch.zeros(3, 4)          # 0으로 채운 3x4 텐서
ones = torch.ones(2, 3)            # 1로 채운 2x3 텐서
rand = torch.rand(2, 2)            # 0~1 균등 분포
randn = torch.randn(2, 2)          # 표준 정규 분포

print(f"zeros (3x4):\n{zeros}")
print(f"\nones (2x3):\n{ones}")
print(f"\nrand (0~1 균등):\n{rand}")
print(f"\nrandn (정규 분포):\n{randn}")

In [ ]:
# 3. 범위/간격 텐서
arange = torch.arange(0, 10, 2)    # 0부터 10 미만, 간격 2
linspace = torch.linspace(0, 1, 5) # 0~1을 5등분

print(f"arange(0, 10, 2): {arange}")
print(f"linspace(0, 1, 5): {linspace}")

In [ ]:
# 4. 기존 텐서와 같은 shape/dtype으로 생성
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)

zeros_like = torch.zeros_like(x)   # x와 같은 shape, 0으로 채움
ones_like = torch.ones_like(x)     # x와 같은 shape, 1로 채움
rand_like = torch.rand_like(x)     # x와 같은 shape, 랜덤

print(f"원본 x:\n{x}")
print(f"zeros_like:\n{zeros_like}")
print(f"ones_like:\n{ones_like}")
print(f"rand_like:\n{rand_like}")

### 실무 예시: 딥러닝 입력 데이터 초기화

In [ ]:
# 배치 데이터 시뮬레이션 (batch_size=32, features=10)
batch_size = 32
num_features = 10

# 입력 데이터 (정규 분포)
X_batch = torch.randn(batch_size, num_features)

# 레이블 (0 또는 1)
y_batch = torch.randint(0, 2, (batch_size,))

print(f"입력 데이터 shape: {X_batch.shape}")
print(f"레이블 shape: {y_batch.shape}")
print(f"\n첫 5개 샘플의 처음 3개 특성:\n{X_batch[:5, :3]}")
print(f"\n첫 10개 레이블: {y_batch[:10]}")

---

## 1.3 텐서 속성

### shape, dtype, device

텐서의 3가지 핵심 속성:
- **shape**: 각 차원의 크기
- **dtype**: 데이터 타입
- **device**: CPU 또는 GPU

In [ ]:
# 텐서 생성
t = torch.randn(3, 4, 5)

# 속성 확인
print(f"Shape: {t.shape}")       # torch.Size([3, 4, 5])
print(f"Size: {t.size()}")       # shape와 동일
print(f"Dtype: {t.dtype}")       # torch.float32 (기본값)
print(f"Device: {t.device}")     # cpu

In [ ]:
# 데이터 타입 지정
int_tensor = torch.tensor([1, 2, 3], dtype=torch.int64)
float_tensor = torch.tensor([1, 2, 3], dtype=torch.float32)
bool_tensor = torch.tensor([True, False, True], dtype=torch.bool)

print(f"int64: {int_tensor}, dtype: {int_tensor.dtype}")
print(f"float32: {float_tensor}, dtype: {float_tensor.dtype}")
print(f"bool: {bool_tensor}, dtype: {bool_tensor.dtype}")

In [ ]:
# 데이터 타입 변환
x = torch.tensor([1, 2, 3])
print(f"원본: {x}, dtype: {x.dtype}")

# float으로 변환
x_float = x.float()  # 또는 x.to(torch.float32)
print(f"float: {x_float}, dtype: {x_float.dtype}")

# double로 변환
x_double = x.double()  # 또는 x.to(torch.float64)
print(f"double: {x_double}, dtype: {x_double.dtype}")

### 실무 예시: 딥러닝에서 자주 사용되는 dtype

| dtype | 용도 | 메모리 |
|-------|-----|--------|
| float32 | 기본 학습 | 4 bytes |
| float16 | 혼합 정밀도 학습 | 2 bytes |
| int64 | 인덱스, 레이블 | 8 bytes |
| bool | 마스크 | 1 byte |

---

## 1.4 GPU 가속 (CUDA)

### to() 메서드로 device 변경

In [ ]:
# device 설정 (GPU 있으면 cuda, 없으면 cpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 device: {device}")

In [ ]:
# 텐서를 device로 이동
x = torch.randn(3, 3)
print(f"원본 device: {x.device}")

# GPU로 이동 (GPU 있는 경우)
x_device = x.to(device)
print(f"이동 후 device: {x_device.device}")

In [ ]:
# 텐서 생성 시 바로 device 지정
y = torch.randn(3, 3, device=device)
print(f"직접 생성: {y.device}")

# CPU로 다시 이동 (결과 확인용)
y_cpu = y.cpu()
print(f"CPU로 이동: {y_cpu.device}")

### 주의: device가 다른 텐서끼리 연산 불가!

In [ ]:
# device 불일치 오류 예시
a = torch.randn(3, 3)  # CPU
b = torch.randn(3, 3, device=device)  # GPU (있는 경우)

if a.device != b.device:
    print(f"주의: a({a.device})와 b({b.device})는 device가 다릅니다!")
    print("연산 전에 같은 device로 이동해야 합니다.")
    
    # 같은 device로 이동 후 연산
    a_device = a.to(device)
    result = a_device + b
    print(f"연산 성공! 결과 shape: {result.shape}")
else:
    result = a + b
    print(f"연산 성공! 결과 shape: {result.shape}")

---

## 1.5 NumPy와 PyTorch 변환

### 양방향 변환

In [ ]:
import numpy as np

# NumPy -> PyTorch
np_array = np.array([[1, 2, 3], [4, 5, 6]])
tensor_from_np = torch.from_numpy(np_array)

print(f"NumPy 배열:\n{np_array}")
print(f"PyTorch 텐서:\n{tensor_from_np}")
print(f"dtype: {tensor_from_np.dtype}")

In [ ]:
# PyTorch -> NumPy
tensor = torch.randn(2, 3)
np_from_tensor = tensor.numpy()

print(f"PyTorch 텐서:\n{tensor}")
print(f"NumPy 배열:\n{np_from_tensor}")
print(f"type: {type(np_from_tensor)}")

### 주의: 메모리 공유!

In [ ]:
# from_numpy는 메모리를 공유합니다!
np_arr = np.array([1, 2, 3], dtype=np.float32)
tensor = torch.from_numpy(np_arr)

print(f"변환 전 NumPy: {np_arr}")
print(f"변환 전 Tensor: {tensor}")

# NumPy 배열 수정
np_arr[0] = 100

print(f"\n수정 후 NumPy: {np_arr}")
print(f"수정 후 Tensor: {tensor}  # 같이 변경됨!")

In [ ]:
# 복사본 만들기 (메모리 공유 방지)
np_arr = np.array([1, 2, 3], dtype=np.float32)
tensor = torch.tensor(np_arr)  # tensor()는 복사본 생성

np_arr[0] = 100

print(f"NumPy: {np_arr}")
print(f"Tensor: {tensor}  # 독립적")

---

## 1.6 텐서 연산

### 기본 산술 연산

In [ ]:
a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

# 요소별 연산
print(f"a + b:\n{a + b}")
print(f"\na - b:\n{a - b}")
print(f"\na * b (요소별):\n{a * b}")
print(f"\na / b:\n{a / b}")

<img class="txc-formula" src="https://t1.daumcdn.net/cfile/tistory/25111F4D58333A5B07" width="481" height="202">

In [ ]:
# 행렬 곱 (Matrix Multiplication)
# 방법 1: torch.matmul()
matmul_result = torch.matmul(a, b)
print(f"matmul(a, b):\n{matmul_result}")


In [ ]:
# 방법 2: @ 연산자 (Python 3.5+)
at_result = a @ b
print(f"\na @ b:\n{at_result}")

In [ ]:
# 방법 3: tensor.mm() (2D only)
mm_result = a.mm(b)
print(f"\na.mm(b):\n{mm_result}")

### 집계 연산

In [ ]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)
print(f"텐서 x:\n{x}")

# 전체 집계
print(f"\nsum: {x.sum()}")
print(f"mean: {x.mean()}")
print(f"max: {x.max()}")
print(f"min: {x.min()}")

In [ ]:
# 축(dim) 기준 집계
print(f"행 방향 합 (dim=0): {x.sum(dim=0)}")
print(f"열 방향 합 (dim=1): {x.sum(dim=1)}")

In [ ]:
# argmax: 최댓값의 인덱스
print(f"\n전체 argmax: {x.argmax()}")
print(f"열 방향 argmax (dim=1): {x.argmax(dim=1)}")

### 텐서 형태 변경

In [ ]:
x = torch.arange(12)
print(f"원본: {x}, shape: {x.shape}")

In [ ]:
# reshape: 형태 변경
x_3x4 = x.reshape(3, 4)
print(f"\nreshape(3, 4):\n{x_3x4}")

In [ ]:
# view: reshape와 유사 (연속 메모리 필요)
x_2x6 = x.view(2, 6)
print(f"\nview(2, 6):\n{x_2x6}")

In [ ]:
# -1: 자동 계산
x_auto = x.reshape(4, -1)  # 4 x ? -> 4 x 3
print(f"\nreshape(4, -1):\n{x_auto}")

In [ ]:
# 차원 추가/제거
x = torch.tensor([1, 2, 3])
print(f"원본 shape: {x.shape}")

In [ ]:
# unsqueeze: 차원 추가
x_unsq0 = x.unsqueeze(0)  # (3,) -> (1, 3)
x_unsq1 = x.unsqueeze(1)  # (3,) -> (3, 1)
print(f"unsqueeze(0): {x_unsq0.shape}")
print(f"unsqueeze(1): {x_unsq1.shape}")

In [ ]:

# squeeze: 크기 1인 차원 제거
y = torch.zeros(1, 3, 1)
y_sq = y.squeeze()
print(f"\nsqueeze 전: {y.shape}")
print(f"squeeze 후: {y_sq.shape}")

---

## 1.7 텐서 인덱싱과 슬라이싱

### NumPy와 동일한 방식

In [ ]:
x = torch.tensor([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12]])
print(f"텐서 x:\n{x}")

In [ ]:
# 기본 인덱싱
print(f"x[0]: {x[0]}")
print(f"x[0, 0]: {x[0, 0]}")
print(f"x[-1]: {x[-1]}")

In [ ]:
# 슬라이싱
print(f"x[:2]: {x[:2]}")
print(f"x[:, 1:3]:\n{x[:, 1:3]}")
print(f"x[1:, 2:]:\n{x[1:, 2:]}")

In [ ]:
# 조건 인덱싱 (Boolean Indexing)
mask = x > 5
print(f"마스크 (x > 5):\n{mask}")
print(f"\n5보다 큰 값: {x[mask]}")

In [ ]:
# 팬시 인덱싱
indices = torch.tensor([0, 2])
print(f"x[indices] (0, 2번 행):\n{x[indices]}")

### 실무 예시: 배치에서 특정 샘플 추출

In [ ]:
# 배치 데이터 (batch_size=8, features=5)
batch = torch.randn(8, 5)
labels = torch.tensor([0, 1, 0, 1, 0, 1, 1, 0])

print(f"배치 shape: {batch.shape}")
print(f"레이블: {labels}")

# 클래스 1인 샘플만 추출
class1_samples = batch[labels == 1]
print(f"\n클래스 1 샘플 shape: {class1_samples.shape}")
print(f"클래스 1 샘플:\n{class1_samples}")

---

# Part 2: 심화

---

## 2.1 Autograd (자동 미분)

### 역전파의 핵심

**Autograd**: 텐서 연산의 그래디언트(미분값)를 자동으로 계산

```
순전파 (Forward):  x -> f(x) -> y -> g(y) -> z
역전파 (Backward): dz/dx <- dz/dy <- dz/dz=1
```

In [ ]:
# requires_grad=True: 그래디언트 추적 활성화
x = torch.tensor([2.0, 3.0], requires_grad=True)
print(f"x: {x}")
print(f"requires_grad: {x.requires_grad}")

In [ ]:
# 연산 수행 (계산 그래프 생성)
y = x ** 2 + 3 * x  # y = x^2 + 3x
print(f"y = x^2 + 3x = {y}")

In [ ]:

# 스칼라로 변환 (backward는 스칼라에 대해 호출)
z = y.sum()  # z = y[0] + y[1]
print(f"z = sum(y) = {z}")

In [ ]:
# backward(): 그래디언트 계산
z.backward()

# x.grad: dz/dx
# y = x^2 + 3x -> dy/dx = 2x + 3
# x = [2, 3] -> dy/dx = [7, 9]
print(f"dz/dx = {x.grad}")
print(f"수동 계산: 2*{x.data} + 3 = {2*x.data + 3}")

### 그래디언트 누적 주의!

In [ ]:
# 그래디언트는 누적됩니다!
x = torch.tensor([1.0], requires_grad=True)

# 첫 번째 backward
y1 = x * 2
y1.backward()
print(f"첫 번째 backward 후 grad: {x.grad}")


In [ ]:

# 두 번째 backward (누적됨!)
y2 = x * 3
y2.backward()
print(f"두 번째 backward 후 grad: {x.grad}  # 2 + 3 = 5")

In [ ]:
# 해결: 매 iteration마다 grad 초기화
x = torch.tensor([1.0], requires_grad=True)

for i in range(3):
    # 그래디언트 초기화
    if x.grad is not None:
        x.grad.zero_()
    
    y = x * (i + 1)
    y.backward()
    print(f"Iteration {i+1}: grad = {x.grad}")

### 실무 예시: 간단한 경사 하강법

In [ ]:
# 목표: y = 2x + 1의 기울기(w)와 절편(b) 찾기
# 데이터 생성
torch.manual_seed(42)
X = torch.linspace(-1, 1, 20).reshape(-1, 1)  # (20, 1)
y_true = 2 * X + 1 + 0.1 * torch.randn_like(X)  # y = 2x + 1 + noise

# 학습할 파라미터 (초기값)
w = torch.tensor([0.0], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

learning_rate = 0.1

print("경사 하강법으로 w=2, b=1 찾기")
print("="*40)

for epoch in range(100):
    # 순전파: 예측
    y_pred = w * X + b
    
    # 손실 계산: MSE
    loss = ((y_pred - y_true) ** 2).mean()
    
    # 역전파: 그래디언트 계산
    loss.backward()
    
    # 파라미터 업데이트 (no_grad 안에서!)
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
    
    # 그래디언트 초기화
    w.grad.zero_()
    b.grad.zero_()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch:3d}: loss={loss.item():.4f}, w={w.item():.4f}, b={b.item():.4f}")

print(f"\n최종 결과: w={w.item():.4f} (목표: 2.0), b={b.item():.4f} (목표: 1.0)")

---

## 2.2 nn.Module로 신경망 정의

### nn.Module 기초

PyTorch의 모든 신경망은 `nn.Module`을 상속받습니다.

In [ ]:
import torch.nn as nn

# 간단한 선형 모델
class LinearModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        # 레이어 정의
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        # 순전파 정의
        return self.linear(x)

# 모델 생성
model = LinearModel(input_dim=10, output_dim=1)
print(model)

In [ ]:
# 모델 파라미터 확인
print("모델 파라미터:")
for name, param in model.named_parameters():
    print(f"  {name}: shape={param.shape}")

In [ ]:
# 순전파 테스트
x = torch.randn(5, 10)  # 배치 5개, 특성 10개
output = model(x)       # forward() 자동 호출
print(f"입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")
print(f"출력:\n{output}")

### 다층 퍼셉트론 (MLP)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        # 레이어 정의
        self.fc1 = nn.Linear(input_dim, hidden_dim)    # 입력 -> 은닉
        self.relu = nn.ReLU()                          # 활성화 함수
        self.fc2 = nn.Linear(hidden_dim, output_dim)   # 은닉 -> 출력
    
    def forward(self, x):
        x = self.fc1(x)      # 선형 변환
        x = self.relu(x)     # 활성화
        x = self.fc2(x)      # 선형 변환
        return x

# MLP 생성
mlp = MLP(input_dim=10, hidden_dim=32, output_dim=2)
print(mlp)

In [ ]:
# nn.Sequential로 더 간단하게
mlp_sequential = nn.Sequential(
    nn.Linear(10, 32),
    nn.ReLU(),
    nn.Linear(32, 2)
)
print(mlp_sequential)

In [ ]:
# 테스트
x = torch.randn(4, 10)
output = mlp_sequential(x)
print(f"입력: {x.shape}")
print(f"출력: {output.shape}")
print(f"출력 값:\n{output}")

---

## 2.3 Dataset과 DataLoader

### Dataset 클래스

데이터를 관리하는 추상 클래스. `__len__`과 `__getitem__` 구현 필요.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 데이터 생성
X_data = torch.randn(100, 5)  # 100개 샘플, 5개 특성
y_data = torch.randint(0, 2, (100,))  # 이진 레이블

# Dataset 생성
dataset = SimpleDataset(X_data, y_data)
print(f"데이터셋 크기: {len(dataset)}")
print(f"첫 번째 샘플: X={dataset[0][0].shape}, y={dataset[0][1]}")

### DataLoader: 배치 처리

In [ ]:
# DataLoader 생성
dataloader = DataLoader(
    dataset,
    batch_size=16,   # 배치 크기
    shuffle=True,    # 셔플 여부
    drop_last=True   # 마지막 불완전한 배치 버림
)

print(f"배치 수: {len(dataloader)}")

# 배치 순회
for batch_idx, (X_batch, y_batch) in enumerate(dataloader):
    if batch_idx < 3:  # 처음 3개 배치만
        print(f"Batch {batch_idx}: X={X_batch.shape}, y={y_batch.shape}")

---

## 2.4 학습 루프 (Training Loop)

### 전체 흐름

```python
for epoch in range(epochs):
    for X_batch, y_batch in dataloader:
        # 1. 순전파
        output = model(X_batch)
        loss = criterion(output, y_batch)
        
        # 2. 역전파
        optimizer.zero_grad()  # 그래디언트 초기화
        loss.backward()        # 그래디언트 계산
        optimizer.step()       # 파라미터 업데이트
```

In [ ]:
# 선형 회귀 예제: y = 2x + 1 학습
torch.manual_seed(42)

# 데이터 생성
X_train = torch.linspace(-2, 2, 100).reshape(-1, 1)
y_train = 2 * X_train + 1 + 0.1 * torch.randn_like(X_train)

# Dataset, DataLoader
train_dataset = SimpleDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 모델, 손실 함수, 옵티마이저
model = nn.Linear(1, 1)  # 입력 1, 출력 1
criterion = nn.MSELoss()  # 평균 제곱 오차
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

print("학습 전 파라미터:")
print(f"  weight: {model.weight.data.item():.4f}")
print(f"  bias: {model.bias.data.item():.4f}")

In [ ]:
# 학습 루프
epochs = 50
losses = []

for epoch in range(epochs):
    epoch_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        # 1. 순전파
        output = model(X_batch)
        loss = criterion(output, y_batch)
        
        # 2. 역전파
        optimizer.zero_grad()  # 그래디언트 초기화
        loss.backward()        # 그래디언트 계산
        optimizer.step()       # 파라미터 업데이트
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}: Loss={avg_loss:.4f}")

print("\n학습 후 파라미터:")
print(f"  weight: {model.weight.data.item():.4f} (목표: 2.0)")
print(f"  bias: {model.bias.data.item():.4f} (목표: 1.0)")

In [ ]:
# 학습 곡선 시각화 (Plotly)
import plotly.express as px
import pandas as pd

loss_df = pd.DataFrame({
    'Epoch': range(epochs),
    'Loss': losses
})

fig = px.line(loss_df, x='Epoch', y='Loss', title='학습 손실 곡선')
fig.update_layout(
    xaxis_title='Epoch',
    yaxis_title='MSE Loss',
    template='plotly_white'
)
fig.show()

### 실무 예시: 이진 분류 모델

In [ ]:
# California Housing 데이터로 고가/저가 분류
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
# 데이터 로드
housing = fetch_california_housing()
X = housing.data

median_price = np.median(housing.target)
y = (housing.target > median_price).astype(int)  # 고가: 1, 저가: 0

# 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 텐서 변환
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

print(f"훈련 데이터: {X_train_t.shape}")
print(f"테스트 데이터: {X_test_t.shape}")

In [ ]:
# 이진 분류 모델
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # 확률 출력
        )
    
    def forward(self, x):
        return self.model(x)

# 모델 생성
model = BinaryClassifier(input_dim=8)
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(model)

In [ ]:
# DataLoader
train_dataset = SimpleDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 학습
epochs = 30
train_losses = []

for epoch in range(epochs):
    model.train()  # 학습 모드
    epoch_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        output = model(X_batch)
        loss = criterion(output, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    train_losses.append(epoch_loss / len(train_loader))
    
    if epoch % 5 == 0:
        print(f"Epoch {epoch:3d}: Loss={train_losses[-1]:.4f}")

In [ ]:
# 평가
model.eval()  # 평가 모드
with torch.no_grad():  # 그래디언트 계산 비활성화
    y_pred_proba = model(X_test_t)
    y_pred = (y_pred_proba > 0.5).float()
    
    accuracy = (y_pred == y_test_t).float().mean()
    print(f"\n테스트 정확도: {accuracy.item():.2%}")

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. 텐서 생성하기 (기본)

**문제**: 다음 조건에 맞는 텐서를 생성하세요.

1. 3x4 크기의 모든 요소가 1인 텐서
2. 0부터 9까지의 정수 텐서
3. 2x3 크기의 표준 정규 분포 랜덤 텐서

In [ ]:
import torch

# 여기에 코드를 작성하세요


### Q2. 텐서 속성 확인 (기본)

**문제**: 아래 텐서의 shape, dtype, device를 출력하세요.

In [ ]:
import torch

tensor = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)

# 여기에 코드를 작성하세요


### Q3. 텐서 연산 (기본)

**문제**: 두 행렬 A와 B의 행렬 곱을 계산하세요.

```python
A = [[1, 2], [3, 4]]
B = [[5, 6], [7, 8]]
```

In [ ]:
import torch

A = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
B = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

# 여기에 코드를 작성하세요


### Q4. NumPy 변환 (응용)

**문제**: NumPy 배열을 PyTorch 텐서로 변환하고, 다시 NumPy로 변환하세요. 메모리 공유 여부를 확인하세요.

In [ ]:
import numpy as np
import torch

np_array = np.array([1.0, 2.0, 3.0])

# 여기에 코드를 작성하세요


### Q5. GPU 이동 (응용)

**문제**: 텐서를 GPU로 이동하고 (GPU가 없으면 CPU 유지), device를 출력하세요.

In [ ]:
import torch

x = torch.randn(3, 3)

# 여기에 코드를 작성하세요


### Q6. 텐서 인덱싱 (응용)

**문제**: 아래 텐서에서 5보다 큰 값만 추출하세요.

In [ ]:
import torch

x = torch.tensor([[1, 8, 3], [4, 2, 9], [7, 6, 5]])

# 여기에 코드를 작성하세요


### Q7. Autograd 이해 (복합)

**문제**: y = x^3 + 2x^2 + 1에서 x=2일 때 dy/dx를 Autograd로 계산하세요.

힌트: dy/dx = 3x^2 + 4x

In [ ]:
import torch

x = torch.tensor([2.0], requires_grad=True)

# 여기에 코드를 작성하세요


### Q8. nn.Module 정의 (복합)

**문제**: 3개의 은닉층을 가진 MLP를 정의하세요.

- 입력: 10
- 은닉층: 64 -> 32 -> 16
- 출력: 2
- 활성화: ReLU

In [ ]:
import torch
import torch.nn as nn

# 여기에 코드를 작성하세요


### Q9. Dataset과 DataLoader (종합)

**문제**: 1000개의 랜덤 샘플(특성 5개)과 레이블로 Dataset을 만들고, batch_size=32인 DataLoader를 생성하세요. 첫 번째 배치를 출력하세요.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# 여기에 코드를 작성하세요


### Q10. 학습 루프 작성 (종합)

**문제**: 아래 데이터로 선형 회귀 모델을 학습하세요.

요구사항:
1. y = 3x - 2의 관계를 학습
2. nn.Linear 모델 사용
3. MSE 손실, SGD 옵티마이저
4. 100 에포크 학습 후 weight와 bias 출력

In [ ]:
import torch
import torch.nn as nn

# 데이터 생성
torch.manual_seed(42)
X = torch.linspace(-1, 1, 100).reshape(-1, 1)
y = 3 * X - 2 + 0.1 * torch.randn_like(X)

# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 함수 | 실무 활용 |
|-----|----------|----------|
| 텐서 생성 | torch.tensor(), zeros(), ones(), rand() | 데이터 초기화 |
| 속성 | shape, dtype, device | 디버깅, 호환성 확인 |
| NumPy 변환 | from_numpy(), .numpy() | 데이터 전처리 연동 |
| GPU 이동 | .to(device), .cuda(), .cpu() | 학습 가속 |
| 인덱싱 | 슬라이싱, Boolean indexing | 배치 추출, 필터링 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 메서드 | 언제 사용? |
|-----|-----------|----------|
| Autograd | requires_grad=True, backward() | 그래디언트 자동 계산 |
| nn.Module | __init__(), forward() | 신경망 정의 |
| Dataset | __len__(), __getitem__() | 데이터 관리 |
| DataLoader | batch_size, shuffle | 배치 처리 |

### 학습 루프 핵심 패턴

```python
for epoch in range(epochs):
    for X_batch, y_batch in dataloader:
        output = model(X_batch)           # 순전파
        loss = criterion(output, y_batch) # 손실 계산
        
        optimizer.zero_grad()  # 그래디언트 초기화
        loss.backward()        # 역전파
        optimizer.step()       # 파라미터 업데이트
```

### 실무 팁

1. **device 일관성**: 모든 텐서와 모델이 같은 device에 있는지 확인
2. **grad 초기화**: 매 iteration마다 `optimizer.zero_grad()` 필수
3. **eval() 모드**: 평가 시 `model.eval()` + `torch.no_grad()`
4. **dtype 주의**: 연산 전 dtype 일치 확인
5. **메모리 관리**: 불필요한 텐서는 `del`로 삭제